# Import thư viện


In [ ]:
from datasets import load_dataset, Audio ,concatenate_datasets , load_from_disk 
from transformers import Wav2Vec2Processor, Wav2Vec2ForSequenceClassification, TrainingArguments, Trainer , Wav2Vec2FeatureExtractor
import torch

In [ ]:
import os
import librosa
from tqdm import tqdm
import random
import json
from collections import defaultdict
import noisereduce as nr
import soundfile as sf
from audiomentations import Compose, AddGaussianNoise, TimeStretch, Gain
import torchaudio

In [ ]:
import numpy as np
import evaluate
from dataclasses import dataclass
from typing import Dict, List, Optional, Union

In [ ]:
import shutil
import glob

In [3]:
# --- Load metadata.jsonl ---
dataset = load_dataset("json", data_files="metadata.jsonl", split="train")

# --- Gán cột audio chuẩn ---
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

# --- Định nghĩa label mapping ---
languages = sorted(list(set(dataset["language"])))
label2id = {lang: i for i, lang in enumerate(languages)}
id2label = {i: lang for lang, i in label2id.items()}

print("Ngôn ngữ:", label2id)

Ngôn ngữ: {'en': 0, 'ja': 1, 'ko': 2, 'th': 3, 'vi': 4, 'zh': 5}


# Check dataset

In [ ]:
root_dir = "datasets/raw/LID"

# 🔹 Danh sách các mã ngôn ngữ của bạn
languages = ["en", "vi", "zh", "ja", "ko", "th"]
print(languages)
count_file ,count_time , count_error_sr , count_error_dim = 0,0,0,0
max , min = 5,5 
threshold = 16000
dict_time= {}
for lang in languages:
        lang_path = os.path.join(root_dir, lang)
        # Duyệt qua các file trong thư mục ngôn ngữ
        count_file = 0 
        count_time = 0
        for file in tqdm(os.listdir(lang_path)):
            if file.endswith((".wav", ".flac", ".mp3")):
                count_file += 1
                y, sr = librosa.load(os.path.join(lang_path, file), sr=None)
                count_time += librosa.get_duration(y=y, sr=sr)
                if librosa.get_duration(y=y, sr=sr) > max:
                    max = librosa.get_duration(y=y, sr=sr)
                if librosa.get_duration(y=y, sr=sr) < min:
                    min = librosa.get_duration(y=y, sr=sr)
                if sr != threshold:
                    count_error_sr += 1
                    print(f"Sampling rate không phù hợp: {sr} Hz {file}")
                if y.ndim > 1:
                    count_error_dim += 1
                    print(f"Số kênh không phù hợp: {y.ndim}  {file}")
        print(f"Số lượng file âm thanh trong {lang}: {count_file}")
        print(f"Tổng thời gian âm thanh trong {lang}: {count_time} giây")
        dict_time[lang] = count_time
        print(f"Thời gian max: {max} giây")
        print(f"Thời gian min: {min} giây")
        print(f"Số lượng file lỗi sampling rate: {count_error_sr}")
        print(f"Số lượng file lỗi số kênh: {count_error_dim}")

# Preprocessing Audio

1. Mỗi audio có cùng sampling rate (thường 16kHz).

2. Độ dài audio không quá dài (giảm tải cho GPU).

3. Tín hiệu được chuẩn hóa về biên độ (volume, độ ồn, khoảng lặng).

4. Đảm bảo mỗi mẫu có label hợp lệ.

5. Tăng cường dữ liệu gồm noise, speed , gain cho tập train

## Split data and Denoise 30% of Folder Train and Augment the remaining 10% of the training set

In [9]:

def split_data_with_denoise(
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    denoise_ratio_in_train=0.3,
    augment_ratio_in_clean=0.1,
):
    root_dir = "datasets/raw/LID"
    os.makedirs("datasets/train", exist_ok=True)
    os.makedirs("datasets/val", exist_ok=True)
    os.makedirs("datasets/test", exist_ok=True)

    # Output metadata
    output_json_train = "datasets/train/metadata_train.jsonl"
    output_json_val = "datasets/val/metadata_val.jsonl"
    output_json_test = "datasets/test/metadata_test.jsonl"

    # Xóa file cũ nếu có
    for file in [output_json_train, output_json_val, output_json_test]:
        if os.path.exists(file):
            os.remove(file)

    # Tổng thời lượng mỗi ngôn ngữ
    total_durations = {
        "en": 175658.64012500033,
        "ja": 202501.0694375005,
        "ko": 274672.68950000097,
        "th": 218886.2407500015,
        "vi": 230769.86056249903,
        "zh": 157739.02868750054,
    }

    # Ngưỡng chia theo tỉ lệ
    dict_train = {lang: total_durations[lang] * train_ratio for lang in total_durations}
    dict_val = {lang: total_durations[lang] * val_ratio for lang in total_durations}
    dict_test = {lang: total_durations[lang] * test_ratio for lang in total_durations}
    dict_denoise = {
        lang: dict_train[lang] * denoise_ratio_in_train for lang in total_durations
    }

    # augment pipeline
    augmentations = {
        "noise": Compose([
            AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=1.0)
    ]),
        "speed": Compose([
            TimeStretch(min_rate=0.9, max_rate=1.1, p=1.0)
    ]),
        "gain": Compose([
            Gain(min_gain_db=-3.0, max_gain_db=3.0, p=1.0)
    ]),
}

    # Duyệt qua từng ngôn ngữ
    for lang in total_durations.keys():
        lang_path = os.path.join(root_dir, lang)
        if not os.path.isdir(lang_path):
            print(f"⚠️ Không tìm thấy thư mục: {lang_path}")
            continue

        files = [
            f for f in os.listdir(lang_path) if f.endswith((".wav", ".flac", ".mp3"))
        ]
        random.shuffle(files)

        count_time = 0
        denoise_time = 0
        augment_time = 0

        for file in tqdm(files, desc=f"🎧 {lang}"):
            audio_path = os.path.join(lang_path, file)

            try:
                duration = librosa.get_duration(path=audio_path)
            except Exception as e:
                print(f"⚠️ Lỗi khi đọc {audio_path}: {e}")
                continue

            count_time += duration

            # Train split
            if count_time <= dict_train[lang]:
                output_file = output_json_train
                file_type = "clean"
                augment_type = "none"

                # ✅ Denoise (30%)
                if denoise_time < dict_denoise[lang]:
                    try:
                        y, sr = librosa.load(audio_path, sr=None)
                        reduced = nr.reduce_noise(y=y, sr=sr)
                        denoise_dir = f"datasets/train/denoise/{lang}"
                        os.makedirs(denoise_dir, exist_ok=True)
                        denoise_path = os.path.join(denoise_dir, f"denoise_{file}")
                        sf.write(denoise_path, reduced, sr)
                        audio_path = denoise_path
                        file_type = "denoise"
                        denoise_time += duration
                    except Exception as e:
                        print(f"❌ Lỗi giảm nhiễu {audio_path}: {e}")

                # ✅ Augment 10% trong phần clean (chỉ khi không denoise)
                elif random.random() < augment_ratio_in_clean:
                    try:
                        y, sr = librosa.load(audio_path, sr=None)
                        aug_choice = random.choice(list(augmentations.keys()))
                        y_aug = augmentations[aug_choice](samples=y, sample_rate=sr)

                        aug_dir = f"datasets/train/augment/{lang}"
                        os.makedirs(aug_dir, exist_ok=True)
                        aug_path = os.path.join(aug_dir, f"aug_{file}")
                        sf.write(aug_path, y_aug, sr)

                        # Ghi metadata augment riêng
                        json_line_aug = {
                            "audio": aug_path.replace("\\", "/"),
                            "language": lang,
                            "duration": duration,
                            "type": "clean",
                            "augment": aug_choice,
                        }
                        with open(output_json_train, "a", encoding="utf-8") as f:
                            f.write(json.dumps(json_line_aug, ensure_ascii=False) + "\n")

                        augment_time += duration
                        augment_type = aug_choice
                    except Exception as e:
                        print(f"❌ Lỗi augment {audio_path}: {e}")

            # Validation split
            elif count_time <= dict_train[lang] + dict_val[lang]:
                output_file = output_json_val
                file_type = "clean"
                augment_type = "none"

            # Test split
            else:
                output_file = output_json_test
                file_type = "clean"
                augment_type = "none"

            # ✅ Ghi metadata chính (dù có augment hay không)
            json_line = {
                "audio": audio_path.replace("\\", "/"),
                "language": lang,
                "duration": duration,
                "type": file_type,
                "augment": augment_type,
            }

            with open(output_file, "a", encoding="utf-8") as f:
                f.write(json.dumps(json_line, ensure_ascii=False) + "\n")

        print(
            f"🗣️ {lang}: train={dict_train[lang]:.1f}s (denoise={dict_denoise[lang]:.1f}s), "
            f"val={dict_val[lang]:.1f}s, test={dict_test[lang]:.1f}s | augment thêm {augment_time/3600:.2f}h"
        )

    print("✅ Hoàn tất chia và xử lý dữ liệu.")


# ---- Gọi hàm ----
if __name__ == "__main__":
    split_data_with_denoise()


🎧 en: 100%|██████████| 15860/15860 [37:57<00:00,  6.96it/s] 


🗣️ en: train=140526.9s (denoise=42158.1s), val=17565.9s, test=17565.9s | augment thêm 2.73h


🎧 ja: 100%|██████████| 22307/22307 [56:52<00:00,  6.54it/s]  


🗣️ ja: train=162000.9s (denoise=48600.3s), val=20250.1s, test=20250.1s | augment thêm 3.24h


🎧 ko: 100%|██████████| 30088/30088 [1:20:21<00:00,  6.24it/s]  


...

🗣️ th: train=175109.0s (denoise=52532.7s), val=21888.6s, test=21888.6s | augment thêm 3.25h


🎧 vi: 100%|██████████| 27295/27295 [1:13:43<00:00,  6.17it/s]  


🗣️ vi: train=184615.9s (denoise=55384.8s), val=23077.0s, test=23077.0s | augment thêm 3.65h


🎧 zh: 100%|██████████| 16293/16293 [45:13<00:00,  6.00it/s] 

🗣️ zh: train=126191.2s (denoise=37857.4s), val=15773.9s, test=15773.9s | augment thêm 2.49h
✅ Hoàn tất chia và xử lý dữ liệu.


## Normalize Audio

In [3]:
def resample_audio(audio_path, target_sr=16000):
    try:
        audio, sr = librosa.load(audio_path, sr=target_sr , mono=True)
        return audio, sr
    except Exception as e:
        print(f"Error loading {audio_path}: {e}")
        return None, None

def normalize_audio(audio):
    return librosa.util.normalize(audio)



In [6]:
def normalize_from_metadata(metadata_path, save_inplace=True):
    # Đọc toàn bộ metadata
    with open(metadata_path, "r", encoding="utf-8") as f:
        lines = [json.loads(line.strip()) for line in f]
    print(f"📄 Tổng số file cần chuẩn hóa: {len(lines)}")
    
    for item in tqdm(lines, desc = "Normalizing"):
        audio_path = item["audio"]

        if not os.path.exists(audio_path):
            print(f"⚠️ Không tìm thấy file: {audio_path}")
            continue
        
        try:
            # Đọc và chuẩn hóa
            y, sr = librosa.load(audio_path, sr=None)
            y_norm = normalize_audio(y)

            if save_inplace:
                # Ghi đè lại file cũ
                sf.write(audio_path, y_norm, sr)
            else:
                # Lưu ra thư mục normalized/
                norm_dir = os.path.join(os.path.dirname(audio_path), "normalized")
                os.makedirs(norm_dir, exist_ok=True)
                norm_path = os.path.join(norm_dir, os.path.basename(audio_path))
                sf.write(norm_path, y_norm, sr)
                # Cập nhật metadata nếu cần
                item["audio"] = norm_path.replace("\\", "/")

        except Exception as e:
            print(f"❌ Lỗi xử lý {audio_path}: {e}")
        
normalize_from_metadata("datasets/train/metadata_train.jsonl")    

📄 Tổng số file cần chuẩn hóa: 117670


Normalizing: 100%|██████████| 117670/117670 [7:08:56<00:00,  4.57it/s]  


# Audio Standardization

* File jsonl sau bước tiền xử lí ở trước sẽ có dạng như bên dưới:

```json
{"audio": "datasets/train/denoise/en/denoise_28Y6RJik15g__U__S102---0994.460-0999.480.wav", "language": "en", "duration": 5.02, "type": "denoise", "augment": "none"}
{"audio": "datasets/train/denoise/en/denoise_HYtn-ogxziE__U__S121---1248.270-1265.120.wav", "language": "en", "duration": 16.85, "type": "denoise", "augment": "none"}
```

* Ta sẽ ánh xạ sang file chứa vector và nhãn của audio, file mới sẽ là float32

* Ta sẽ gộp tất các ngôn ngữ lại với nhau và trộn lẫn để chúng học được phân biệt ngay từ đầu và tập val cũng thế để đánh giá trong quá trình train 

* Và sẽ nén chúng lại thành các .arrow 

## Khởi tạo đường dẫn và hàm preprocess

In [8]:

# Giả sử bạn đã chạy pip install soundfile và các thư viện khác
# LƯU Ý: VÌ BẠN ĐANG CHẠY TRÊN LOCAL, HÃY CÀI ĐẶT CÁC THƯ VIỆN CẦN THIẾT.

# ======================================================
# 1️⃣ CẤU HÌNH LOCAL
# ======================================================
LANGUAGES = ["vi", "en", "ja", "ko", "zh", "th"]
MODEL_NAME = "facebook/wav2vec2-large-xlsr-53"

# Cấu hình Local của bạn
LOCAL_BASE_DIR = "datasets" 
LOCAL_CACHE_DIR = f"{LOCAL_BASE_DIR}/cache/wav2vec2" 


JSONL_TEST_LOCAL = f"{LOCAL_BASE_DIR}/test/metadata_test.jsonl"
SAVE_MAPPED_DATA_DIR = f"{LOCAL_BASE_DIR}/cache"


# ======================================================
# 2️⃣ HÀM TIỀN XỬ LÝ (SỬ DỤNG SOUNDFILE)
# ======================================================
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(MODEL_NAME)

def preprocess(batch):
    """Tải audio bằng soundfile và trích xuất đặc trưng."""
    
    audio_path = batch["audio"]
    
    # 💡 LOGIC ĐIỀU CHỈNH ĐƯỜNG DẪN LOCAL:
    # Nếu file audio trong JSONL chỉ chứa đường dẫn tương đối (ví dụ: "audio/vi/001.wav")
    # Chúng ta phải nối nó với thư mục datasets gốc.
    # if not os.path.isabs(audio_path):
    #     # Nối LOCAL_BASE_DIR (datasets) với đường dẫn tương đối trong JSONL
    #     # Kết quả: datasets/audio/vi/001.wav (Đúng với cấu trúc local)
    #     audio_path = os.path.join(LOCAL_BASE_DIR, audio_path) 

    if not os.path.exists(audio_path):
        # Đây là lỗi phổ biến nhất, cần kiểm tra kỹ cấu trúc thư mục local
        raise FileNotFoundError(f"❌ Không tìm thấy file âm thanh: {audio_path}. Kiểm tra đường dẫn JSONL.")

    # Load audio bằng soundfile (Ổn định hơn torchaudio trên Windows)
    # data: numpy array, sampling_rate: int
    data, sampling_rate = soundfile.read(audio_path)
    
    # Xử lý âm thanh Stereo (2D array) -> Mono (1D array)
    if data.ndim > 1:
        speech_array = data.mean(axis=1) # Lấy trung bình hai kênh
    else:
        speech_array = data # Giữ nguyên mono

    # Xử lý dữ liệu cho Wav2Vec2 (Input là numpy array)
    batch["input_values"] = feature_extractor(
        speech_array,
        sampling_rate=sampling_rate,
        return_tensors="pt"
    ).input_values[0]

    batch["label"] = LANGUAGES.index(batch["language"])
    return batch

## Map danh sách tập test 

In [ ]:

try:

        
    ds_test = load_dataset("json", data_files=JSONL_TEST_LOCAL, split="train")
    # Trộn (Shuffle)
    print(f"Đang trộn (shuffle) {len(ds_test)} mẫu tổng...")
    # (Bây giờ nó sẽ dùng CACHE_DIR và không bị lỗi)
    ds_test = ds_test.shuffle(seed=42)
    print(f"Bắt đầu map ds_val ({len(ds_test)} mẫu)...")
    ds_test = ds_test.map(preprocess, batched=False)
    
    print(f"Map ds_val ({len(ds_test)} mẫu) thành công ...")
    

    # 4. Lưu ra Disk (vào /kaggle/working/) - Phần này đã ĐÚNG
    os.makedirs(SAVE_MAPPED_DATA_DIR, exist_ok=True)
    test_global_save_path = os.path.join(SAVE_MAPPED_DATA_DIR, "GLOBAL_test")
    
    print(f"Đang lưu tập GLOBAL_val vào: {test_global_save_path}")
    ds_test.save_to_disk(test_global_save_path)
    
    print(f"\n🎉 THÀNH CÔNG! Đã lưu Tập Validation Toàn cầu tại:")
    print(f"{test_global_save_path}")

except Exception as e:
    print(f"❌ LỖI TRONG QUÁ TRÌNH TẠO GLOBAL_test: {e}")


🌍 BẮT ĐẦU TẠO TẬP TEST TOÀN CẦU (GLOBAL_test)


Generating train split: 0 examples [00:00, ? examples/s]

Đang trộn (shuffle) 13918 mẫu tổng...
Bắt đầu map ds_val (13918 mẫu)...


Map:   0%|          | 0/13918 [00:00<?, ? examples/s]

Map ds_val (13918 mẫu) thành công ...
Đang lưu tập GLOBAL_val vào: datasets/cache\GLOBAL_test


Saving the dataset (0/17 shards):   0%|          | 0/13918 [00:00<?, ? examples/s]


🎉 THÀNH CÔNG! Đã lưu Tập Validation Toàn cầu tại:
datasets/cache\GLOBAL_test


## Map danh sách tập train

In [ ]:
DATA_PATH = "datasets"
LANGUAGES = ["vi", "en", "ja", "ko", "zh", "th"]
# 1. Tạo một danh sách rỗng
all_train_datasets = []
print("Bắt đầu tải từng thư mục .arrow riêng lẻ...")

try:
    # 2. Vòng lặp và Tải từng thư mục
    for lang in LANGUAGES:
        # Đường dẫn đến thư mục .arrow của ngôn ngữ (ví dụ: /.../vi_train/vi_train)
        lang_train_path_nested = os.path.join(DATA_PATH, f"{lang}_train", f"{lang}_train")
        print(f"Đang tải: {lang_train_path_nested}")
        
        ds_lang_train = load_from_disk(lang_train_path_nested)
        ds_lang_train = ds_lang_train.map(lambda x: x)
        # 3. Thêm vào danh sách
        all_train_datasets.append(ds_lang_train)
        print(f"✅ Đã tải và cache lại {len(ds_lang_train)} mẫu cho '{lang}'.")

    # 4. Gộp (Concatenate)
    print("\nĐang gộp (concatenate) tất cả 6 dataset lại...")
    ds_train_global = concatenate_datasets(all_train_datasets)
    
    print("\n🎉 THÀNH CÔNG!")
    print(f"Dataset toàn cầu đã được load và gộp, tổng cộng có {len(ds_train_global)} mẫu.")
    
    # (Bây giờ bạn có thể dùng 'ds_train_global' để shuffle, save_to_disk, hoặc train)
    # Ví dụ:
    print("Đang trộn (shuffle)...")
    ds_train_global = ds_train_global.shuffle(seed=42)
    print("Đang lưu ra disk...")
    ds_train_global.save_to_disk(r"D:\HuggingFace_Cache\cache")

except Exception as e:
    print(f"❌ LỖI KHI ĐANG TẢI: {e}")

## Map danh sách tập val

In [ ]:
DATA_PATH = "datasets"
LANGUAGES = ["vi", "en", "ja", "ko", "zh", "th"]
# 1. Tạo một danh sách rỗng
all_val_datasets = []
print("Bắt đầu tải từng thư mục .arrow riêng lẻ...")

try:
    # 2. Vòng lặp và Tải từng thư mục
    for lang in LANGUAGES:
        # Đường dẫn đến thư mục .arrow của ngôn ngữ (ví dụ: /.../vi_train/vi_train)
        lang_val_path_nested = os.path.join(DATA_PATH, f"{lang}_val", f"{lang}_val")
        print(f"Đang tải: {lang_val_path_nested}")
        
        ds_lang_val = load_from_disk(lang_val_path_nested)
        ds_lang_val = ds_lang_val.map(lambda x: x)
        # 3. Thêm vào danh sách
        all_val_datasets.append(ds_lang_val)
        print(f"✅ Đã tải và cache lại {len(ds_lang_val)} mẫu cho '{lang}'.")

    # 4. Gộp (Concatenate)
    print("\nĐang gộp (concatenate) tất cả 6 dataset lại...")
    ds_val_global = concatenate_datasets(all_val_datasets)
    
    print("\n🎉 THÀNH CÔNG!")
    print(f"Dataset toàn cầu đã được load và gộp, tổng cộng có {len(ds_val_global)} mẫu.")
    
    # (Bây giờ bạn có thể dùng 'ds_val_global' để shuffle, save_to_disk, hoặc train)
    # Ví dụ:
    print("Đang trộn (shuffle)...")
    ds_val_global = ds_val_global.shuffle(seed=42)
    print("Đang lưu ra disk...")
    ds_val_global.save_to_disk(r"D:\HuggingFace_Cache\cache")

except Exception as e:
    print(f"❌ LỖI KHI ĐANG TẢI: {e}")

# Train

In [ ]:
def compute_metrics(p):
    """Tính Accuracy cho quá trình đánh giá (Evaluation)."""
    predictions = np.argmax(p.predictions, axis=1)
    return {"accuracy": (predictions == p.label_ids).mean()}

In [ ]:
@dataclass
class DataCollatorSpeechV2:
    feature_extractor: Wav2Vec2FeatureExtractor
    padding: Union[bool, str] = "longest"
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["label"] for feature in features]
        batch = self.feature_extractor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        batch["labels"] = torch.tensor(label_features, dtype=torch.long)
        return batch

In [ ]:
print("\n=======================================================")
print("🚀 BẮT ĐẦU HUẤN LUYỆN HỖN HỢP (MIXED TRAINING 100%)")
print("=======================================================")
SAVE_CHECKPOINT_DIR = r"models\Checkpoint"
SAVE_MODEL_DIR = r"models\saved"
try:
    # --- 1. Tải bằng Memory-Mapping (KHÔNG TỐN RAM) ---
    print(f"Tải GLOBAL_train (77GB) từ: {TRAIN_PATH}")
    ds_train = load_from_disk(TRAIN_PATH)
    print(f"Tải GLOBAL_val từ: {VAL_PATH}")
    ds_val = load_from_disk(VAL_PATH)
    
    print(f"✅ Tải dữ liệu thành công (qua Memory-Map).")
    print(f"   Train: {len(ds_train)} | Val: {len(ds_val)}")

    # --- 2. Tải Model Gốc ---
    model = Wav2Vec2ForSequenceClassification.from_pretrained(
        MODEL_NAME, 
        num_labels=len(LANGUAGES)
    )
    
    # --- 3. Cấu hình Arguments (Sử dụng batch 16 cho L40s) ---
    training_args = TrainingArguments(
        output_dir=SAVE_CHECKPOINT_DIR,
        per_device_train_batch_size=16, # (Như bạn nói, sẽ chiếm ~42GB VRAM)
        gradient_accumulation_steps=1, 
        dataloader_num_workers=2,       
        eval_strategy="epoch",         
        save_strategy="epoch",
        num_train_epochs=3,             # Train 3 epoch
        learning_rate=5e-5, 
        fp16=True,                      # Dùng fp16 cho L40s/T4
        save_total_limit=2,             
        logging_steps=100,             
        report_to="none",
        load_best_model_at_end=True,    # Tự động chọn model tốt nhất
        metric_for_best_model="accuracy",
        greater_is_better=True,
    )

    trainer = Trainer(
        model=model, 
        args=training_args, 
        train_dataset=ds_train, 
        eval_dataset=ds_val,   
        tokenizer=feature_extractor,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    # --- 4. Huấn luyện ---
    print("Bắt đầu huấn luyện...")
    trainer.train() 
    print("Huấn luyện hoàn tất.")

    # --- 5. Lưu Model Cuối cùng ---
    trainer.save_model(SAVE_MODEL_DIR) 
    feature_extractor.save_pretrained(SAVE_MODEL_DIR) 
    print(f"✅ Đã lưu model cuối cùng (tốt nhất) tại: {SAVE_MODEL_DIR}")

except Exception as e:
    print(f"❌ LỖI TRONG QUÁ TRÌNH HUẤN LUYỆN: {e}")

# Evaluation

In [ ]:
MODEL_TO_TEST_PATH = r"models\Checkpoint"

In [15]:
metric = evaluate.load("accuracy")

In [ ]:
# === 2. TẢI MODEL VÀ DỮ LIỆU ===

print(f"Đang tải model từ: {MODEL_TO_TEST_PATH}")
model = Wav2Vec2ForSequenceClassification.from_pretrained(MODEL_TO_TEST_PATH)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(MODEL_TO_TEST_PATH)

print(f"Đang tải dữ liệu validation từ: {VAL_DATA_PATH}")
ds_val = load_from_disk(VAL_DATA_PATH)

print("✅ Tải model và dữ liệu thành công.")

# === 3. CHẠY ĐÁNH GIÁ (PREDICT) ===

# Khởi tạo Data Collator
data_collator = DataCollatorSpeechV2(feature_extractor=feature_extractor)

# Cấu hình Trainer tối thiểu
test_args = TrainingArguments(
    output_dir=r"output", # Thư mục tạm
    per_device_eval_batch_size=16,          # Tăng batch size để test nhanh
    report_to="none"
)

# Khởi tạo Trainer
tester = Trainer(
    model=model,
    args=test_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# --- CHẠY DỰ ĐOÁN ---
print("\nBắt đầu chạy dự đoán trên tập GLOBAL_val...")
results = tester.predict(ds_val) # Sử dụng ds_val ở đây

print("\n--- KẾT QUẢ ĐÁNH GIÁ (TRÊN GLOBAL_val) ---")
# In ra các chỉ số (ví dụ: test_loss, test_accuracy)
# Kết quả sẽ có dạng {'predict_loss': ..., 'predict_accuracy': ...}
print(results.metrics)

In [ ]:
# === 2. TẢI MODEL VÀ DỮ LIỆU ===

print(f"Đang tải model từ: {MODEL_TO_TEST_PATH}")
model = Wav2Vec2ForSequenceClassification.from_pretrained(MODEL_TO_TEST_PATH)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(MODEL_TO_TEST_PATH)

print(f"Đang tải dữ liệu validation từ: {TEST_DATA_PATH}")
ds_test = load_from_disk(TEST_DATA_PATH)

print("✅ Tải model và dữ liệu thành công.")

# === 3. CHẠY ĐÁNH GIÁ (PREDICT) ===

# Khởi tạo Data Collator
data_collator = DataCollatorSpeechV2(feature_extractor=feature_extractor)

In [ ]:

# ====================================================
# 1. CẤU HÌNH THƯ MỤC LỖI
# ====================================================
# Thư mục để lưu các file audio nhận diện sai
ERROR_DIR = "./error"

# Xóa thư mục cũ (nếu có) để tạo mới cho sạch
if os.path.exists(ERROR_DIR):
    shutil.rmtree(ERROR_DIR)
os.makedirs(ERROR_DIR, exist_ok=True)

print(f"📁 Các file lỗi sẽ được lưu vào: {ERROR_DIR}")

# ====================================================
# 2. CHẠY ĐÁNH GIÁ (PREDICT)
# ====================================================

# Cấu hình Trainer tối thiểu (như cũ)
test_args = TrainingArguments(
    output_dir="./temp_test", 
    per_device_eval_batch_size=16,          
    report_to="none",
    dataloader_num_workers=2,       
    fp16=True if torch.cuda.is_available() else False
)

tester = Trainer(
    model=model,
    args=test_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("\n🚀 Bắt đầu chạy dự đoán trên tập test...")
results = tester.predict(ds_test) 

# ====================================================
# 3. PHÂN TÍCH VÀ LƯU FILE LỖI
# ====================================================
print("\n🔍 Đang phân tích các mẫu sai...")

# Lấy kết quả
logits = results.predictions
pred_ids = np.argmax(logits, axis=1) # Nhãn dự đoán (0, 1, 2...)
true_ids = results.label_ids         # Nhãn thật

# Danh sách ghi log
error_logs = []

In [ ]:
CSV_OUTPUT_PATH = "error/error_report.csv"
LANGUAGES = ["vi", "en", "ja", "ko", "zh", "th"]
print(f"Tổng số mẫu kiểm tra: {len(pred_ids)}")

for idx, (pred, true) in enumerate(zip(pred_ids, true_ids)):
    if pred != true:
        # === ĐÂY LÀ MỘT MẪU SAI ===
        try:
            # 1. Lấy tên ngôn ngữ
            true_lang = LANGUAGES[true]
            pred_lang = LANGUAGES[pred]
            
            # 2. Lấy thông tin từ Dataset (File Arrow)
            item = ds_test[idx]
            
            # 3. Trích xuất đường dẫn file gốc (Xử lý linh hoạt)
            audio_path = "N/A"
            if 'audio' in item:
                val = item['audio']
                if isinstance(val, str):
                    audio_path = val
                elif isinstance(val, dict):
                    # HuggingFace datasets thường lưu audio dưới dạng dict {'path': ..., 'array': ...}
                    audio_path = val.get('path', 'Unknown Path')
            
            # 4. Tính độ tin cậy (Confidence) của dự đoán sai
            # Lấy vector logit của mẫu hiện tại
            sample_logits = logits[idx]
            # Dùng Softmax để chuyển sang xác suất
            probs = np.exp(sample_logits) / np.sum(np.exp(sample_logits))
            confidence = probs[pred] # Độ tự tin vào cái sai
            true_score = probs[true] # Độ tự tin vào cái đúng (thường sẽ thấp)

            # 5. Ghi vào log
            error_record = {
                "Index": idx,
                "Audio_Path": audio_path,  # Đường dẫn gốc để tra cứu
                "True_Label": true_lang,   # Nhãn đúng
                "Pred_Label": pred_lang,   # Nhãn mô hình đoán sai
                "Confidence": round(float(confidence), 4), # Mức độ tự tin khi sai
                "True_Label_Score": round(float(true_score), 4) # Điểm cho nhãn đúng
            }
            
            error_logs.append(error_record)

        except Exception as e:
            print(f"⚠️ Lỗi trích xuất tại dòng {idx}: {e}")

# ====================================================
# 4. TỔNG KẾT VÀ XUẤT BÁO CÁO CSV
# ====================================================
print(f"\n✅ Đã hoàn tất phân tích! Tìm thấy {len(error_logs)} mẫu sai.")

if len(error_logs) > 0:
    # Tạo DataFrame
    df_errors = pd.DataFrame(error_logs)
    
    # Sắp xếp: Đưa những lỗi mà mô hình "tự tin nhất" lên đầu (Nguy hiểm nhất)
    df_errors = df_errors.sort_values(by="Confidence", ascending=False)
    
    # Lưu ra CSV
    csv_path = os.path.join(ERROR_DIR, "error_report.csv")
    df_errors.to_csv(csv_path, index=False, encoding='utf-8-sig')
    
    print(f"📄 Báo cáo chi tiết đã lưu tại: {csv_path}")
    
    # Hiển thị 5 lỗi điển hình nhất
    print("\n--- 5 Mẫu lỗi có độ tin cậy cao nhất (Model sai nhưng rất tự tin) ---")
    # Chọn các cột quan trọng để hiển thị
    display_cols = ['Audio_Path', 'True_Label', 'Pred_Label', 'Confidence']
    print(df_errors.head(5)[display_cols].to_string(index=False))
else:
    print("🎉 Chúc mừng! Mô hình dự đoán đúng 100% trên tập này (hoặc không tìm thấy lỗi).")

print("\n--- KẾT QUẢ METRICS TỔNG QUÁT ---")
print(results.metrics)